Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = './data/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [8]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights = tf.Variable(
      tf.truncated_normal([image_size * image_size, num_labels]), name='weight')
    biases = tf.Variable(tf.zeros([num_labels]), name='bias')

    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
      tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run this computation and iterate:

In [9]:
num_steps = 801

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
            / predictions.shape[0])

with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases. 
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
      _, l, predictions = session.run([optimizer, loss, train_prediction])
      if (step % 100 == 0):
        print('Loss at step %d: %f' % (step, l))
        print('Training accuracy: %.1f%%' % accuracy(
          predictions, train_labels[:train_subset, :]))
        # Calling .eval() on valid_prediction is basically like calling run(), but
        # just to get that one numpy array. Note that it recomputes all its graph
        # dependencies.
        print('Validation accuracy: %.1f%%' % accuracy(
          valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 17.239510
Training accuracy: 8.0%
Validation accuracy: 9.6%
Loss at step 100: 2.315452
Training accuracy: 72.2%
Validation accuracy: 70.8%
Loss at step 200: 1.878820
Training accuracy: 74.8%
Validation accuracy: 73.0%
Loss at step 300: 1.630932
Training accuracy: 76.1%
Validation accuracy: 73.7%
Loss at step 400: 1.460065
Training accuracy: 77.1%
Validation accuracy: 74.3%
Loss at step 500: 1.332914
Training accuracy: 77.9%
Validation accuracy: 74.5%
Loss at step 600: 1.233582
Training accuracy: 78.4%
Validation accuracy: 74.8%
Loss at step 700: 1.152950
Training accuracy: 78.9%
Validation accuracy: 75.2%
Loss at step 800: 1.085737
Training accuracy: 79.3%
Validation accuracy: 75.3%
Test accuracy: 82.7%


Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [10]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run it:

In [11]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 16.540693
Minibatch accuracy: 14.8%
Validation accuracy: 15.5%
Minibatch loss at step 500: 1.288214
Minibatch accuracy: 78.1%
Validation accuracy: 75.1%
Minibatch loss at step 1000: 1.536904
Minibatch accuracy: 76.6%
Validation accuracy: 76.3%
Minibatch loss at step 1500: 0.769245
Minibatch accuracy: 81.2%
Validation accuracy: 77.4%
Minibatch loss at step 2000: 0.700666
Minibatch accuracy: 85.9%
Validation accuracy: 77.7%
Minibatch loss at step 2500: 1.068400
Minibatch accuracy: 78.1%
Validation accuracy: 78.8%
Minibatch loss at step 3000: 0.968185
Minibatch accuracy: 76.6%
Validation accuracy: 78.6%
Test accuracy: 86.2%


---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

In [4]:
x = tf.placeholder(tf.float32, shape=[None, 28*28], name='x')
y = tf.placeholder(tf.int32, shape=[None, 10], name='y')
layer1_weight = tf.get_variable('layer1_weight', shape=[28*28, 1024], 
                                initializer=tf.truncated_normal_initializer(stddev=1024/(28*28)))
layer1_bias = tf.get_variable('layer1_bias', shape=[1024], initializer=tf.zeros_initializer())
layer2_weight = tf.get_variable('layer2_weight', shape=[1024, 10],
                               initializer=tf.truncated_normal_initializer(stddev=10/1024))
layer2_bias = tf.get_variable('layer2_bias', shape=[10], initializer=tf.zeros_initializer())

layer1_out = tf.nn.relu(tf.matmul(x, layer1_weight) + layer1_bias, name='layer1_output')
logits = tf.matmul(layer1_out, layer2_weight) + layer2_bias

In [5]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits), 0)
correct = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1)), tf.float32), 0)
optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

In [6]:
total_step = 3000
batch_size = 128
step_skip = 100
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    running_loss = 0.0
    running_acc = 0.0
    for step in range(total_step):
        offset = (step * batch_size) % (train_dataset.shape[0] - batch_size)
        _, batch_loss, batch_correct = sess.run([optimizer, loss, correct],
                                               feed_dict={x: train_dataset[offset:offset+batch_size],
                                                          y: train_labels[offset:offset+batch_size]})
        running_loss += batch_loss
        running_acc += batch_correct
        if (step + 1) % step_skip == 0:
            print('step {} loss:{:.6f}, Acc:{:.6f}'.format((step+1), running_loss / step_skip, running_acc / step_skip))
            running_loss = 0.0
            running_acc = 0.0
    test_correct = sess.run(correct, feed_dict={x: test_dataset, y: test_labels})
    print('test acc:{:.6f}'.format(test_correct))

step 100 loss:145.727258, Acc:0.724766
step 200 loss:39.323040, Acc:0.768984
step 300 loss:34.046518, Acc:0.780781
step 400 loss:30.515612, Acc:0.781953
step 500 loss:26.190923, Acc:0.786484
step 600 loss:27.182431, Acc:0.791875
step 700 loss:20.103520, Acc:0.793047
step 800 loss:20.592631, Acc:0.795703
step 900 loss:17.316320, Acc:0.789531
step 1000 loss:16.920420, Acc:0.797344
step 1100 loss:18.473804, Acc:0.791875
step 1200 loss:13.311226, Acc:0.802656
step 1300 loss:14.987766, Acc:0.796953
step 1400 loss:13.530476, Acc:0.798594
step 1500 loss:13.305628, Acc:0.802500
step 1600 loss:12.203854, Acc:0.806016
step 1700 loss:11.306059, Acc:0.809609
step 1800 loss:10.096868, Acc:0.808672
step 1900 loss:10.037216, Acc:0.814844
step 2000 loss:10.019978, Acc:0.813906
step 2100 loss:8.855774, Acc:0.816094
step 2200 loss:8.895795, Acc:0.817891
step 2300 loss:9.435890, Acc:0.812031
step 2400 loss:8.976636, Acc:0.814063
step 2500 loss:7.686096, Acc:0.813594
step 2600 loss:8.808273, Acc:0.814141
